In [ ]:
import logging
import random

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import seaborn as sns
from tqdm import tqdm

from hsurf.hsurf import tilted_algo as tilted_algo
from hsurf import site_selection_strategy
import pylib

random.seed(1)

logging.basicConfig(
    format="[%(funcName)s:%(lineno)d] %(message)s",
)
logger = logging.getLogger()

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
nbm.print_metadata()

pylib.jupyter_hide_toggle(hide=True)


In [ ]:
for surface_size in 8, 16, 32:
    surface_history_df = pylib.site_selection_eval.make_surface_history_df(
        tilted_algo.pick_ingest_site,
        surface_size=surface_size,
        num_generations=min(2**18, tilted_algo.get_ingest_capacity(surface_size)),
        progress_wrap=tqdm,
    ).replace(-1, np.nan)

    candidate_ranks = [
        2**i - 1 for i in range(surface_size.bit_length() - 1, surface_size)
    ]
    maxrank = surface_history_df["rank"].max()
    ranks = [
        rank
        for rank in candidate_ranks
        if rank <= maxrank  # in operator weird, nan?
    ]
    pylib.tee_release(
        pylib.site_selection_viz.site_reservation_at_ranks_heatmap,
        surface_history_df,
        ranks=ranks,
        plotter_=pylib.site_selection_viz.site_reservation_at_rank_stripped_heatmap,
        teeplot_outattrs={"surface-size": surface_size},
        teeplot_subdir="20a",
    )
    plt.show()


In [ ]:
def clear_ax(
    *_args,
    **kwargs,
) -> mpl.axes.Axes:
    kwargs["palette"] = ["white"] * 20
    kwargs["zigwidth"] = 0.0
    kwargs["zigzag"] = False
    return pylib.site_selection_viz.site_reservation_at_rank_stripped_heatmap(
        *_args,
        **kwargs,
    )


In [ ]:
for surface_size in 8, 16, 32:
    surface_history_df = pylib.site_selection_eval.make_surface_history_df(
        tilted_algo.pick_ingest_site,
        surface_size=surface_size,
        num_generations=min(2**18, tilted_algo.get_ingest_capacity(surface_size)),
        progress_wrap=tqdm,
    ).replace(-1, np.nan)

    candidate_ranks = [
        2**i - 1 for i in range(surface_size.bit_length() - 1, surface_size)
    ]
    maxrank = surface_history_df["rank"].max()
    ranks = [
        rank
        for rank in candidate_ranks
        if rank <= maxrank  # in operator weird, nan?
    ]
    pylib.tee_release(
        pylib.site_selection_viz.site_reservation_at_ranks_heatmap,
        surface_history_df,
        ranks=ranks,
        plotter_=clear_ax,
        teeplot_outattrs={"surface-size": surface_size, "plotter": "clear_ax"},
        teeplot_subdir="20a",
    )
    plt.show()


## Visualize Tilted Criterion Satisfaction


In [ ]:
mpl.rcParams.update(
    {
        "figure.figsize": (6, 1.5),
        "font.family": "serif",
    },
)


In [ ]:
for surface_size in 8, 16, 32:
    surface_history_df = pylib.site_selection_eval.make_surface_history_df(
        tilted_algo.pick_ingest_site,
        surface_size=surface_size,
        num_generations=min(2**surface_size - 1, 2**16),
        progress_wrap=tqdm,
    )
    criteria_df = pylib.site_selection_eval.calc_surface_history_criteria(
        surface_history_df,
        tilted_algo,
        site_selection_strategy.site_selection_bounds,
        site_selection_strategy.site_selection_criteria,
        enforce_tilted_bound=True,
        progress_wrap=tqdm,
    )
    with pylib.teed_release(
        pylib.site_selection_viz.criterion_satisfaction_lineplot,
        data=criteria_df[
            ~criteria_df["kind"].isin(["naive lower bound"])
        ],
        x="rank",
        y="tilted criterion",
        hue="kind",
        surface_size=surface_size,
        teeplot_outinclude="surface_size",
        teeplot_subdir="20a",
    ) as ax:
        sns.move_legend(
            ax, "lower center",
            bbox_to_anchor=(.45, 1),
            ncol=2,
            title=None,
            frameon=False,
            fontsize="large",
        )
        ax.set_xlabel("Time ($T$)")
        ax.set_ylabel("Gap Size Ratio\n($\\mathsf{cost\_tilted}$)")
